In [1]:
%%writefile mapper.py

from __future__ import division
import sys
import re
from collections import Counter


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode
path = 'stop_words_en.txt'
with open(path) as f:
    stop_words = f.read()
stop_words = stop_words.split()
for line in sys.stdin:
    i=0
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        article_id = int(article_id)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    words = [x.lower() for x in words if x.lower() not in stop_words]
    uniq_words = set(words)
    words_in_articles = len(words)
    counts = Counter(words)
    for word, word_count in counts.items():
        tf = word_count/words_in_articles
        print "%s\t%d\t%f" % (word, article_id,tf)


Overwriting mapper.py


In [2]:
%%writefile reducer.py
from __future__ import division
import sys
from math import log



current_key = None
article_dict = dict()

for line in sys.stdin:
    try:
        key, article_id,tf = line.strip().split('\t',2)
        article_id = int(article_id)
        tf = float(tf)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            idf = 1/log(1+len(article_dict))
            for article_id_0, tf in article_dict.iteritems():
                tf_idf = tf*idf
                print "%s\t%d\t%f" % (current_key,article_id_0,tf_idf)
        article_dict = dict()
        current_key = key
    article_dict[article_id] = tf
    
if current_key:
    print "%s\t%d\t%f" % (current_key,article_id_0,tf_idf)

Overwriting reducer.py


In [3]:
%%bash

OUT_DIR="coursera_mr_task2"$(date +"%s%6N")
NUM_REDUCERS=8

# Stub code for your job
# rm $LOGS 
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="TF Assignment" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input  /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 
# ... \
# -reducer "python reducer.py" \

# yarn jar /usr/local/hadoop-2.8.1/hadoop-streaming.jar \
#     -D mapreduce.job.name="Stop Words" \
#     -D mapreduce.job.reduces=${NUM_REDUCERS} \
#     -files mapper.py,reducer.py,stop_words_en.txt \
#     -mapper "python mapper.py" \
#     -reducer "python reducer.py" \
#     -input  /data/wiki/en_articles_part \
#     -output ${OUT_DIR} > /dev/null 2> $LOGS

hdfs dfs -cat ${OUT_DIR}/* | grep -P  "labor\t12\t" |cut -f3

20/03/08 21:33:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/03/08 21:33:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/03/08 21:33:23 INFO mapred.FileInputFormat: Total input files to process : 1
20/03/08 21:33:23 INFO mapreduce.JobSubmitter: number of splits:2
20/03/08 21:33:23 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1583697892150_0020
20/03/08 21:33:23 INFO impl.YarnClientImpl: Submitted application application_1583697892150_0020
20/03/08 21:33:23 INFO mapreduce.Job: The url to track the job: http://1a00d52b991a:8088/proxy/application_1583697892150_0020/
20/03/08 21:33:23 INFO mapreduce.Job: Running job: job_1583697892150_0020
20/03/08 21:33:29 INFO mapreduce.Job: Job job_1583697892150_0020 running in uber mode : false
20/03/08 21:33:29 INFO mapreduce.Job:  map 0% reduce 0%
20/03/08 21:33:47 INFO mapreduce.Job:  map 5% reduce 0%
20/03/08 21:33:53 INFO mapreduce.Job:  map 7% reduce 0%
20/03/08 21:33:59 INFO ma

# 